In [1]:
!apt-get update

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,305 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,599 kB]
Get:13 https://cloud.r-project.org/bin/linux/ubuntu j

In [2]:
!apt-get -y install nvidia-cuda-toolkit

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libaccinj64-11.5 libatk-wrapper-java libatk-wrapper-java-jni
  libbabeltrace1 libcub-dev libcublas11 libcublaslt11 libcudart11.0 libcufft10 libcufftw10
  libcuinj64-11.5 libcupti-dev libcupti-doc libcupti11.5 libcurand10 libcusolver11 libcusolvermg11
  libcusparse11 libdebuginfod-common libdebuginfod1 libegl-dev libfontenc1 libgail-common libgail18
  libgl-dev libgl1-mesa-dev libgles-dev libgles1 libglvnd-core-dev libglvnd-dev libglx-dev
  libgtk2.0-0 libgtk2.0-bin libgtk2.0-common libipt2 libnppc11 libnppial11 libnppicc11 libnppidei11
  libnppif11 libnppig11 libnppim11 libnppist11 libnppisu11 libnppitc11 libnpps11 libnvblas11
  libnvidia-compute-495 libnvidia-compute-510 libnvidia-compute-525 libnvidia-ml-dev libnvjpeg11
  libnvrtc-builtins11.5 libnvrtc11.2 libnvtoolsext1 libnvvm4 libopengl-dev libr

In [3]:
!nvidia-smi

Tue Dec 26 13:12:54 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [5]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-o_0su_hv
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-o_0su_hv
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4293 sha256=d8752587088c5c993c803802a0e01beeabddd61455014c891266f2452d20a3c8
  Stored in directory: /tmp/pip-ephem-wheel-cache-fxfcoucs/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [6]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cu
#include <stdio.h>
#include <math.h>
#include <string.h>


#define CSC(call) 							\
do { 										\
	cudaError_t status = call;				\
	if (status != cudaSuccess) {																				\
		fprintf(stderr, "ERROR is %s:%d. Message: %s\n", __FILE__, __LINE__, cudaGetErrorString(status));		\
		exit(0);								\
	}											\
} while(0)


typedef unsigned char uchar;


struct vec3 {
    double x;
    double y;
    double z;
};


__host__ __device__ double dot(vec3 a, vec3 b) {
	  return a.x * b.x + a.y * b.y + a.z * b.z;
}

/*
| i  j  k  |
| x1 y1 z1 | = i * (y1 * z2 - z1 * y2) + j * ... + k * ...
| x2 y2 z2 |
*/


__host__ __device__ vec3 prod(vec3 a, vec3 b) {
	  return {a.y * b.z - a.z * b.y, a.z * b.x - a.x * b.z,  a.x * b.y - a.y * b.x};
}


__host__ __device__ vec3 norm(vec3 v) {
	  double l = sqrt(dot(v, v));
	  return {v.x / l, v.y / l, v.z / l};
}


__host__ __device__ vec3 diff(vec3 a, vec3 b) {
	  return {a.x - b.x, a.y - b.y, a.z - b.z};
}


__host__ __device__ vec3 add(vec3 a, vec3 b) {
	  return {a.x + b.x, a.y + b.y, a.z + b.z};
}


__host__ __device__ vec3 mult(vec3 a, vec3 b, vec3 c, vec3 v) {
	  return {a.x * v.x + b.x * v.y + c.x * v.z,
			      a.y * v.x + b.y * v.y + c.y * v.z,
			      a.z * v.x + b.z * v.y + c.z * v.z};
}

__host__ __device__ vec3 mult_num(vec3 a, double num) {
	  return {a.x * num, a.y * num, a.z * num};
}


struct trig {
    vec3 a;
    vec3 b;
    vec3 c;
    uchar4 color;
};


__host__ __device__ uchar4 ray(vec3 pos, vec3 dir, vec3 light_pos, uchar4 light_color, trig* trigs, int trigs_cnt) {
    int k_min = -1;
    double ts_min;
    for (int k = 0; k < trigs_cnt; k++) {
        vec3 e1 = diff(trigs[k].b, trigs[k].a);
        vec3 e2 = diff(trigs[k].c, trigs[k].a);
        vec3 p = prod(dir, e2);
        double div = dot(p, e1);
        if (fabs(div) < 1e-10) {
          continue;
        }
        vec3 t = diff(pos, trigs[k].a);
        double u = dot(p, t) / div;
        if (u < 0.0 || u > 1.0) {
          continue;
        }
        vec3 q = prod(t, e1);
        double v = dot(q, dir) / div;
        if (v < 0.0 || v + u > 1.0) {
          continue;
        }
        double ts = dot(q, e2) / div;
        if (ts < 0.0) {
          continue;
        }
        if (k_min == -1 || ts < ts_min) {
          k_min = k;
          ts_min = ts;
        }
    }

    if (k_min == -1) {
      return {0, 0, 0, 255};
    }

    vec3 new_pos = add(mult_num(dir, ts_min), pos);
    vec3 new_dir = diff(light_pos, new_pos);
    double length = sqrt(dot(new_dir, new_dir));
    new_dir = norm(new_dir);

    for (int k = 0; k < trigs_cnt; k++) {
        vec3 e1 = diff(trigs[k].b, trigs[k].a);
        vec3 e2 = diff(trigs[k].c, trigs[k].a);
        vec3 p = prod(new_dir, e2);
        double div = dot(p, e1);
        if (fabs(div) < 1e-10) {
            continue;
        }
        vec3 t = diff(new_pos, trigs[k].a);
        double u = dot(p, t) / div;
        if (u < 0.0 || u > 1.0) {
            continue;
        }
        vec3 q = prod(t, e1);
        double v = dot(q, new_dir) / div;
        if (v < 0.0 || v + u > 1.0) {
            continue;
        }
        double ts = dot(q, e2) / div;
        if (ts > 0.0 && ts < length && k != k_min) {
            return {0, 0, 0, 255};
        }
    }

    return {trigs[k_min].color.x * light_color.x, trigs[k_min].color.y * light_color.y, trigs[k_min].color.z * light_color.z, 255};
}

__host__ __device__ void render(vec3 pc, vec3 pv, int w, int h, double angle, uchar4 *data, vec3 light_pos, uchar4 light_color, trig* trigs, int trigs_cnt) {
    double dw = 2.0 / (w - 1.0);
    double dh = 2.0 / (h - 1.0);
    double z = 1.0 / tan(angle * M_PI / 360.0);
    vec3 bz = norm(diff(pv, pc));
    vec3 bx = norm(prod(bz, {0.0, 0.0, 1.0}));
    vec3 by = norm(prod(bx, bz));
    for(int i = 0; i < w; i++) {
        for(int j = 0; j < h; j++) {
          vec3 v = {-1.0 + dw * i, (-1.0 + dh * j) * h / w, z};
          vec3 dir = mult(bx, by, bz, v);
          data[(h - 1 - j) * w + i] = ray(pc, norm(dir), light_pos, light_color, trigs, trigs_cnt);
        }
    }
}

__global__ void kernel_render(vec3 pc, vec3 pv, int w, int h, double angle, uchar4 *data, vec3 light_pos, uchar4 light_color, trig* trigs, int trigs_cnt) {
    int idx = blockDim.x * blockIdx.x + threadIdx.x;
    int idy = blockDim.y * blockIdx.y + threadIdx.y;
    int offsetx = blockDim.x * gridDim.x;
    int offsety = blockDim.y * gridDim.y;

    double dw = 2.0 / (w - 1.0);
    double dh = 2.0 / (h - 1.0);
    double z = 1.0 / tan(angle * M_PI / 360.0);
    vec3 bz = norm(diff(pv, pc));
    vec3 bx = norm(prod(bz, {0.0, 0.0, 1.0}));
    vec3 by = norm(prod(bx, bz));
    for (int i = idx; i < w; i += offsetx) {
        for (int j = idy; j < h; j += offsety) {
          vec3 v = {-1.0 + dw * i, (-1.0 + dh * j) * h / w, z};
          vec3 dir = mult(bx, by, bz, v);
          data[(h - 1 - j) * w + i] = ray(pc, norm(dir), light_pos, light_color, trigs, trigs_cnt);
        }
    }
}


__host__ __device__ void ssaa(uchar4* data, uchar4* ssaa_data, int w, int h, int sqrt_rays) {
    for (int x = 0; x < w; ++x) {
        for (int y = 0; y < h; ++y) {
            uint4 tmp = make_uint4(0, 0, 0, 0);
            for (int i = 0; i < sqrt_rays; ++i) {
                for (int j = 0; j < sqrt_rays; ++j) {
                    uchar4 cur_pixel = data[w * sqrt_rays * (y * sqrt_rays + j) + (x * sqrt_rays + i)];
                    tmp.x += cur_pixel.x;
                    tmp.y += cur_pixel.y;
                    tmp.z += cur_pixel.z;
                }
            }
            int rays_per_pixel = sqrt_rays * sqrt_rays;
            ssaa_data[y * w + x] = (uchar4){tmp.x / rays_per_pixel, tmp.y / rays_per_pixel, tmp.z / rays_per_pixel, 255};
        }
    }
}


__global__ void kernel_ssaa(uchar4* data, uchar4* ssaa_data, int w, int h, int sqrt_rays) {
    int idx = blockDim.x * blockIdx.x + threadIdx.x;
    int idy = blockDim.y * blockIdx.y + threadIdx.y;
    int offsetx = blockDim.x * gridDim.x;
    int offsety = blockDim.y * gridDim.y;

    for (int x = idx; x < w; x += offsetx) {
        for (int y = idy; y < h; y += offsety) {
            uint4 tmp = make_uint4(0, 0, 0, 0);
            for (int i = 0; i < sqrt_rays; ++i) {
                for (int j = 0; j < sqrt_rays; ++j) {
                    uchar4 cur_pixel = data[w * sqrt_rays * (y * sqrt_rays + j) + (x * sqrt_rays + i)];
                    tmp.x += cur_pixel.x;
                    tmp.y += cur_pixel.y;
                    tmp.z += cur_pixel.z;
                }
            }
            int rays_per_pixel = sqrt_rays * sqrt_rays;
            ssaa_data[y * w + x] = make_uchar4(tmp.x / rays_per_pixel, tmp.y / rays_per_pixel, tmp.z / rays_per_pixel, 255);
        }
    }
}


void make_floor(vec3 a, vec3 b, vec3 c, vec3 d, uchar4 color, trig* trigs, int i) {
    trigs[i] = {a, b, c, color};
    trigs[i + 1] = {a, c, d, color};
}


void make_tetrahedron(vec3 center, uchar4 color, double r, trig* trigs, int i) {
    double a = r * sqrt(3);
    vec3 vertices[] = {{center.x - a / 2, 0, center.z - a / sqrt(12)}, {center.x, center.y + r, center.z - a / sqrt(12)}, {center.x + a / 2, 0, center.z - a / sqrt(12)}, {center.x, center.y, center.z + r}};
    trigs[i] = {vertices[0], vertices[1], vertices[2], color};
    trigs[i + 1] = {vertices[0], vertices[1], vertices[3], color};
    trigs[i + 2] = {vertices[0], vertices[2], vertices[3], color};
    trigs[i + 3] = {vertices[1], vertices[2], vertices[3], color};
}


void make_hexahedron(vec3 center, uchar4 color, double r, trig* trigs, int i) {
    double a = 2 * r / sqrt(3);
    vec3 first_v = {center.x - a / 2, center.y - a / 2, center.z - a / 2};
    vec3 vertices[] = {
        {first_v.x, first_v.y, first_v.z},
        {first_v.x, first_v.y + a, first_v.z},
        {first_v.x + a, first_v.y + a, first_v.z},
        {first_v.x + a, first_v.y, first_v.z},
        {first_v.x, first_v.y, first_v.z + a},
        {first_v.x, first_v.y + a, first_v.z + a},
        {first_v.x + a, first_v.y + a, first_v.z + a},
        {first_v.x + a, first_v.y, first_v.z + a}
    };
    trigs[i] = {vertices[0], vertices[1], vertices[2], color};
    trigs[i + 1] = {vertices[2], vertices[3], vertices[0], color};
    trigs[i + 2] = {vertices[6], vertices[7], vertices[3], color};
    trigs[i + 3] = {vertices[3], vertices[2], vertices[6], color};
    trigs[i + 4] = {vertices[2], vertices[1], vertices[5], color};
    trigs[i + 5] = {vertices[5], vertices[6], vertices[2], color};
    trigs[i + 6] = {vertices[4], vertices[5], vertices[1], color};
    trigs[i + 7] = {vertices[1], vertices[0], vertices[4], color};
    trigs[i + 8] = {vertices[3], vertices[7], vertices[4], color};
    trigs[i + 9] = {vertices[4], vertices[0], vertices[3], color};
    trigs[i + 10] = {vertices[6], vertices[5], vertices[4], color};
    trigs[i + 11] = {vertices[4], vertices[7], vertices[6], color};
}


void make_dodecahedron(vec3 center, uchar4 color, double r, trig* trigs, int i) {
    double a = (1 + sqrt(5)) / 2;
    double b = 1 / a;
    vec3 vertices[] = {
        {-b, 0, a},
        {b, 0, a},
        {-1, 1, 1},
        {1, 1, 1},
        {1, -1, 1},
        {-1, -1, 1},
        {0, -a, b},
        {0, a, b},
        {-a, -b, 0},
        {-a, b, 0},
        {a, b, 0},
        {a, -b, 0},
        {0, -a, -b},
        {0, a, -b},
        {1, 1, -1},
        {1, -1, -1},
        {-1, -1, -1},
        {-1, 1, -1},
        {b, 0, -a},
        {-b, 0, -a},
    };
    for (int i = 0; i < 20; i++) {
        vertices[i].x = vertices[i].x * r / sqrt(3) + center.x;
        vertices[i].y = vertices[i].y * r / sqrt(3) + center.y;
        vertices[i].z = vertices[i].z * r / sqrt(3) + center.z;
    }
    trigs[i] = {vertices[4], vertices[0], vertices[6], color};
    trigs[i + 1] = {vertices[0], vertices[5], vertices[6], color};
    trigs[i + 2] = {vertices[0], vertices[4], vertices[1], color};
    trigs[i + 3] = {vertices[0], vertices[3], vertices[7], color};
    trigs[i + 4] = {vertices[2], vertices[0], vertices[7], color};
    trigs[i + 5] = {vertices[0], vertices[1], vertices[3], color};
    trigs[i + 6] = {vertices[10], vertices[1], vertices[11], color};
    trigs[i + 7] = {vertices[3], vertices[1], vertices[10], color};
    trigs[i + 8] = {vertices[1], vertices[4], vertices[11], color};
    trigs[i + 9] = {vertices[5], vertices[0], vertices[8], color};
    trigs[i + 10] = {vertices[0], vertices[2], vertices[9], color};
    trigs[i + 11] = {vertices[8], vertices[0], vertices[9], color};
    trigs[i + 12] = {vertices[5], vertices[8], vertices[16], color};
    trigs[i + 13] = {vertices[6], vertices[5], vertices[12], color};
    trigs[i + 14] = {vertices[12], vertices[5], vertices[16], color};
    trigs[i + 15] = {vertices[4], vertices[12], vertices[15], color};
    trigs[i + 16] = {vertices[4], vertices[6], vertices[12], color};
    trigs[i + 17] = {vertices[11], vertices[4], vertices[15], color};
    trigs[i + 18] = {vertices[2], vertices[13], vertices[17], color};
    trigs[i + 19] = {vertices[2], vertices[7], vertices[13], color};
    trigs[i + 20] = {vertices[9], vertices[2], vertices[17], color};
    trigs[i + 21] = {vertices[13], vertices[3], vertices[14], color};
    trigs[i + 22] = {vertices[7], vertices[3], vertices[13], color};
    trigs[i + 23] = {vertices[3], vertices[10], vertices[14], color};
    trigs[i + 24] = {vertices[8], vertices[17], vertices[19], color};
    trigs[i + 25] = {vertices[16], vertices[8], vertices[19], color};
    trigs[i + 26] = {vertices[8], vertices[9], vertices[17], color};
    trigs[i + 27] = {vertices[14], vertices[11], vertices[18], color};
    trigs[i + 28] = {vertices[11], vertices[15], vertices[18], color};
    trigs[i + 29] = {vertices[10], vertices[11], vertices[14], color};
    trigs[i + 30] = {vertices[12], vertices[19], vertices[18], color};
    trigs[i + 31] = {vertices[15], vertices[12], vertices[18], color};
    trigs[i + 32] = {vertices[12], vertices[16], vertices[19], color};
    trigs[i + 33] = {vertices[19], vertices[13], vertices[18], color};
    trigs[i + 34] = {vertices[17], vertices[13], vertices[19], color};
    trigs[i + 35] = {vertices[13], vertices[14], vertices[18], color};
}


void print_default() {
    printf("120\n");
    printf("res/%%d.data\n");
    printf("640 480 120\n");
    printf("7.0 3.0 0.0 2.0 1.0 2.0 6.0 1.0 0.0 0.0\n");
    printf("2.0 0.0 0.0 0.5 0.1 1.0 4.0 1.0 0.0 0.0\n");
    printf("3.0 2.0 0.6 0.3 0.75 0.0 1.0\n");
    printf("0.0 0.0 0.0 0.6 0.25 0.55 1.75\n");
    printf("-3.0 -2.0 0.0 0.0 0.8 0.7 1.5\n");
    printf("-5.0 -5.0 -1.0 -5.0 5.0 -1.0 5.0 5.0 -1.0 5.0 -5.0 -1.0 1.0 0.9 0.35\n");
    printf("-10.0 0.0 12.0 0.4 0.3 0.1\n");
    printf("4\n");
}


int main(int argc, char* argv[]) {
    bool gpu = true;
    if (argc >= 2) {
        if (strcmp(argv[1], "--default") == 0) {
            print_default();
            return 0;
        }
        if (strcmp(argv[1], "--cpu") == 0) {
            gpu = false;
        }
    }

    // ввод данных по заданию
    int frames_number = 120;
    char output_path[256] = "res/%d.data";
    int w = 640, h = 480;
    double angle = 120;
    double r0c = 7.0, z0c = 3.0, phi0c = 0.0, Arc = 2.0, Azc = 1.0, wrc = 2.0, wzc = 6.0, wphic = 1.0, prc = 0.0, pzc = 0.0;
    double r0n = 2.0, z0n = 0.0, phi0n = 0.0, Arn = 0.5, Azn = 0.1, wrn = 1.0, wzn = 4.0, wphin = 1.0, prn = 0.0, pzn = 0.0;
    double center1_x = 3.0, center1_y = 2.0, center1_z = 0.6, color1_x = 0.3, color1_y = 0.75, color1_z = 0.0, r1 = 1.0;
    double center2_x = 0.0, center2_y = 0.0, center2_z = 0.0, color2_x = 0.6, color2_y = 0.25, color2_z = 0.55, r2 = 1.75;
    double center3_x = -3.0, center3_y = -2.0, center3_z = 0.0, color3_x = 0.0, color3_y = 0.8, color3_z = 0.7, r3 = 1.5;
    double floor1_x = -5.0, floor1_y = -5.0, floor1_z = -1.0, floor2_x = -5.0, floor2_y = 5.0, floor2_z = -1.0, floor3_x = 5.0, floor3_y = 5.0, floor3_z = -1.0, floor4_x = 5.0, floor4_y = -5.0, floor4_z = -1.0, floor_color_x = 1.0, floor_color_y = 0.9, floor_color_z = 0.35;
    double light_pos_x = -10.0, light_pos_y = 0.0, light_pos_z = 12.0;
    double light_color_x = 0.4, light_color_y = 0.3, light_color_z = 0.1;
    double sqrt_rays = 4;
    /*
    scanf("%d", &frames_number);
    scanf("%s", output_path);
    scanf("%d %d %lf", &w, &h, &angle);
    scanf("%lf %lf %lf %lf %lf %lf %lf %lf %lf %lf", &r0c, &z0c, &phi0c, &Arc, &Azc, &wrc, &wzc, &wphic, &prc, &pzc);
    scanf("%lf %lf %lf %lf %lf %lf %lf %lf %lf %lf", &r0n, &z0n, &phi0n, &Arn, &Azn, &wrn, &wzn, &wphin, &prn, &pzn);
    scanf("%lf %lf %lf", &center1_x, &center1_y, &center1_z);
    scanf("%lf %lf %lf", &color1_x, &color1_y, &color1_z);
    scanf("%lf", &r1);
    scanf("%lf %lf %lf", &center2_x, &center2_y, &center2_z);
    scanf("%lf %lf %lf", &color2_x, &color2_y, &color2_z);
    scanf("%lf", &r2);
    scanf("%lf %lf %lf", &center3_x, &center3_y, &center3_z);
    scanf("%lf %lf %lf", &color3_x, &color3_y, &color3_z);
    scanf("%lf", &r3);
    scanf("%lf %lf %lf %lf %lf %lf", &floor1_x, &floor1_y, &floor1_z, &floor2_x, &floor2_y, &floor2_z);
    scanf("%lf %lf %lf %lf %lf %lf", &floor3_x, &floor3_y, &floor3_z, &floor4_x, &floor4_y, &floor4_z);
    scanf("%lf %lf %lf", &floor_color_x, &floor_color_y, &floor_color_z);
    scanf("%lf %lf %lf", &light_pos_x, &light_pos_y, &light_pos_z);
    scanf("%lf %lf %lf", &light_color_x, &light_color_y, &light_color_z);
    scanf("%lf", &sqrt_rays);*/

    vec3 light_pos = (vec3){light_pos_x, light_pos_y, light_pos_z};
    uchar4 light_color = (uchar4){light_color_x * 255, light_color_y * 255, light_color_z * 255, 255};

    uchar4* data = (uchar4*)malloc(sizeof(uchar4) * w * h * sqrt_rays * sqrt_rays);
    uchar4* ssaa_data = (uchar4*)malloc(sizeof(uchar4) * w * h);
    uchar4* dev_data;
    uchar4* dev_ssaa_data;
    trig* dev_trigs;
    char buffer[500];
    trig trigs[54];
    make_floor((vec3){floor1_x, floor1_y, floor1_z}, (vec3){floor2_x, floor2_y, floor2_z}, (vec3){floor3_x, floor3_y, floor3_z}, (vec3){floor4_x, floor4_y, floor4_z}, (uchar4){floor_color_x * 255, floor_color_y * 255, floor_color_z * 255, 255}, trigs, 0);
    make_tetrahedron((vec3){center1_x, center1_y, center1_z}, (uchar4){color1_x * 255, color1_y * 255, color1_z * 255, 255}, r1, trigs, 2);
    make_hexahedron((vec3){center2_x, center2_y, center2_z}, (uchar4){color2_x * 255, color2_y * 255, color2_z * 255, 255}, r2, trigs, 6);
    make_dodecahedron((vec3){center3_x, center3_y, center3_z}, (uchar4){color3_x * 255, color3_y * 255, color3_z * 255, 255}, r3, trigs, 18);

    if (gpu) {
        CSC(cudaMalloc(&dev_data, sizeof(uchar4) * w * h * sqrt_rays * sqrt_rays));
        CSC(cudaMalloc(&dev_ssaa_data, sizeof(uchar4) * w * h));
        CSC(cudaMalloc(&dev_trigs, sizeof(trig) * 54));
        CSC(cudaMemcpy(dev_trigs, trigs, sizeof(trig) * 54, cudaMemcpyHostToDevice));
    }

    for (int frame = 0; frame < frames_number; frame++) {
        double t = 2 * M_PI * frame / frames_number;
        vec3 pc, pv;
        double rc = r0c + Arc * sin(wrc * t + prc);
        double zc = z0c + Azc * sin(wzc * t + pzc);
        double phic = phi0c + wphic * t;
        double rn = r0n + Arn * sin(wrn * t + prn);
        double zn = z0n + Azn * sin(wzn * t + pzn);
        double phin = phi0n + wphin * t;
        pc.x = rc * cos(phic);
        pc.y = rc * sin(phic);
        pc.z = zc;
        pv.x = rn * cos(phin);
        pv.y = rn * sin(phin);
        pv.z = zn;

        cudaEvent_t start, stop;
        CSC(cudaEventCreate(&start));
        CSC(cudaEventCreate(&stop));
        CSC(cudaEventRecord(start));

        if (gpu) {
            kernel_render<<<dim3(16, 16), dim3(16, 16)>>>(pc, pv, w * sqrt_rays, h * sqrt_rays, angle, dev_data, light_pos, light_color, dev_trigs, 54);
            CSC(cudaGetLastError());
            kernel_ssaa<<<dim3(16, 16), dim3(16, 16)>>>(dev_data, dev_ssaa_data, w, h, sqrt_rays);
            CSC(cudaGetLastError());
            CSC(cudaMemcpy(ssaa_data, dev_ssaa_data, sizeof(uchar4) * w * h, cudaMemcpyDeviceToHost));
        }
        else {
            render(pc, pv, w * sqrt_rays, h * sqrt_rays, angle, data, light_pos, light_color, trigs, 54);
            ssaa(data, ssaa_data, w, h, sqrt_rays);
        }
        CSC(cudaEventRecord(stop));
        CSC(cudaEventSynchronize(stop));

        float time;
        CSC(cudaEventElapsedTime(&time, start, stop));
        CSC(cudaEventDestroy(start));
        CSC(cudaEventDestroy(stop));

        printf("%d\t%f\t%lf\n", frame + 1, time, w * h * sqrt_rays * sqrt_rays);

        sprintf(buffer, output_path, frame);
        FILE* output_file = fopen(buffer, "w");
        fwrite(&w, sizeof(int), 1, output_file);
        fwrite(&h, sizeof(int), 1, output_file);
        fwrite(ssaa_data, sizeof(uchar4), w * h, output_file);
        fclose(output_file);
    }

    if (gpu) {
        CSC(cudaFree(dev_data));
        CSC(cudaFree(dev_ssaa_data));
    }
    free(data);
    free(ssaa_data);
    return 0;
}

In [24]:
!nvcc cp.cu

cp.cu(141): warning #2361-D: invalid narrowing conversion from "int" to "unsigned char"
      return {trigs[k_min].color.x * light_color.x, trigs[k_min].color.y * light_color.y, trigs[k_min].color.z * light_color.z, 255};
              ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

cp.cu(141): warning #2361-D: invalid narrowing conversion from "int" to "unsigned char"
      return {trigs[k_min].color.x * light_color.x, trigs[k_min].color.y * light_color.y, trigs[k_min].color.z * light_color.z, 255};
                                                    ^

cp.cu(141): warning #2361-D: invalid narrowing conversion from "int" to "unsigned char"
      return {trigs[k_min].color.x * light_color.x, trigs[k_min].color.y * light_color.y, trigs[k_min].color.z * light_color.z, 255};
                                                                                          ^

cp.cu(141): warning #2361-D: invalid narrowing conversion from "int" to "unsigned char"
  

In [22]:
!./a.out --default

120
res/%d.data
640 480 120
7.0 3.0 0.0 2.0 1.0 2.0 6.0 1.0 0.0 0.0
2.0 0.0 0.0 0.5 0.1 1.0 4.0 1.0 0.0 0.0
3.0 2.0 0.6 0.3 0.75 0.0 1.0
0.0 0.0 0.0 0.6 0.25 0.55 1.75
-3.0 -2.0 0.0 0.0 0.8 0.7 1.5
-5.0 -5.0 -1.0 -5.0 5.0 -1.0 5.0 5.0 -1.0 5.0 -5.0 -1.0 1.0 0.9 0.35
-10.0 0.0 12.0 0.4 0.3 0.1
4


In [25]:
!./a.out --gpu

241.410873, 142.610077, 113.499809, 112.418915, 112.331139, 111.409698, 111.091614, 110.772224, 111.453278, 110.170113, 109.517281, 109.177025, 108.694595, 108.332542, 108.675522, 108.571938, 108.908096, 108.678467, 108.748482, 109.146339, 110.017059, 110.365379, 110.544701, 110.481918, 110.914497, 111.345123, 111.804962, 112.349663, 113.384514, 114.248993, 117.962563, 121.649277, 124.727997, 129.896454, 134.958969, 138.510468, 139.734177, 140.377121, 140.654495, 139.012253, 137.090073, 135.110168, 132.984100, 131.523865, 130.404449, 129.689026, 129.762375, 130.932419, 132.723007, 135.116837, 138.547943, 141.421860, 143.835266, 145.336197, 145.278748, 143.614075, 140.111038, 134.762939, 129.508835, 126.487228, 123.850945, 119.119713, 116.873184, 115.862305, 115.078270, 114.728127, 114.555740, 114.041954, 114.788834, 113.334145, 112.854721, 112.110718, 111.802307, 111.888351, 112.112480, 112.139969, 112.490784, 112.707840, 112.373055, 112.373985, 111.386238, 111.497917, 111.538338, 111.

In [27]:
!./a.out --cpu

14487.465820, 14292.223633, 14171.903320, 13982.832031, 14049.304688, 13863.850586, 14561.862305, 14123.495117, 13950.254883, 13857.287109, 13720.309570, 13735.390625, 13780.300781, 13577.224609, 13579.736328, 13603.637695, 13706.013672, 13661.980469, 13523.492188, 13531.312500, 13635.315430, 13823.250000, 13793.672852, 13721.258789, 13834.798828, 13980.214844, 14524.628906, 14411.004883, 14340.933594, 14370.817383, 14518.694336, 14885.343750, 15537.769531, 17236.023438, 17185.648438, 17132.957031, 18037.699219, 17615.531250, 17281.130859, 17826.072266, 16899.869141, 16600.921875, 16554.093750, 17289.035156, 16057.637695, 16079.715820, 15992.751953, 16999.925781, 16174.685547, 16537.474609, 17194.908203, 17863.964844, 17579.191406, 17890.521484, 17937.599609, 17295.496094, 17524.746094, 16779.082031, 15704.738281, 15190.489258, 14865.166016, 14940.565430, 15060.946289, 14486.001953, 14194.615234, 14115.666992, 14238.795898, 14114.172852, 14226.333984, 14014.320312, 14142.625000, 13929.

In [ ]:
!pwd

/content


In [ ]:
!lscpu

Architecture:            x86_64
  CPU op-mode(s):        32-bit, 64-bit
  Address sizes:         46 bits physical, 48 bits virtual
  Byte Order:            Little Endian
CPU(s):                  2
  On-line CPU(s) list:   0,1
Vendor ID:               GenuineIntel
  Model name:            Intel(R) Xeon(R) CPU @ 2.20GHz
    CPU family:          6
    Model:               79
    Thread(s) per core:  2
    Core(s) per socket:  1
    Socket(s):           1
    Stepping:            0
    BogoMIPS:            4399.99
    Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clf
                         lush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_
                         good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fm
                         a cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hyp
                         ervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd i

In [ ]:
!lsb_release -a

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.3 LTS
Release:	22.04
Codename:	jammy
